In [1]:
# ladowanie modulu od unittestow
%load_ext ipython_unittest
from collections import Counter
from geopy import distance
from utils import *
import requests 

In [2]:
%%unittest_main
class MergeTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        reqUsers = requests.get("https://jsonplaceholder.typicode.com/users")
        reqPosts = requests.get("https://jsonplaceholder.typicode.com/posts")
        if [reqUsers.status_code, reqPosts.status_code] != [200, 200]: 
            raise Exception("Error: couldn't retrieve data from the server") 
        cls.posts = reqPosts.json()
        cls.users = reqUsers.json()
        cls.mL = mergeLists(cls.posts, cls.users, "userId")

    def testPostsNotNull(self):
        for i in self.posts:
            for j in i:
                self.assertIsNotNone(j)

    def testUsersNotNull(self):
        for i in self.users:
            for j in i:
                self.assertIsNotNone(j)

    def testMergedProperly(self):
        postKeys = len(self.posts[0].keys())
        userKeys = len(self.users[0].keys())
        self.assertEqual(len(self.mL[0].keys()), 11)

    def testIdsOfPosts(self):
        for i in range(len(self.posts)):
            self.assertEqual(self.posts[i]['id'], i+1)

    def testIdsOfUsers(self):
        for i in range(len(self.users)):
            self.assertEqual(self.users[i]['id'], i+1)

    def testMergeListIds(self):
        for i in range(len(self.mL)):
            self.assertEqual(self.mL[i]['id'], i+1)

    def testMergeListUIds(self):
        userIds = [i+1 for i in range(len(self.users))]
        for i in self.mL:
            self.assertIn(i['userId'], userIds)

    def testPostsPerUserUsernames(self):
        usernames= []
        for i in range(len(self.users)):
            usernames.append(self.users[i]['username'])
        counts = countPostsPerUser(self.mL)
        for i in counts: 
            self.assertIn(counts[i][1], usernames)

Success

........
----------------------------------------------------------------------
Ran 8 tests in 0.284s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

In [3]:
%%unittest_main
class PostsMethodsTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.posts = requests.get("https://jsonplaceholder.typicode.com/posts").json()
        cls.titles = getPostTitles(cls.posts)

    def testTypeOfTitles(self):
        for i in self.titles:
            self.assertIsInstance(i, str)
    
    def testNumberOfTitles(self):
        self.assertEqual(len(self.posts), len(self.titles))

    def testNonUniqueTitles(self):
        title = self.titles.copy()
        title.append(title[0])
        nonUniqueTitle = [key for (key,value) in Counter(title).items() if value > 1]
        self.assertEqual(nonUniqueTitle, [title[0]])

    def testAllTitlesUnique(self):
        title = self.titles.copy()
        nonUniqueTitles = [key for (key,value) in Counter(title).items() if value > 1]
        self.assertEqual(nonUniqueTitles, [])


Success

....
----------------------------------------------------------------------
Ran 4 tests in 0.261s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [4]:
%%unittest_main
class DistanceTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.users = requests.get("https://jsonplaceholder.typicode.com/users").json()
        cls.distances = calculateDistances(cls.users)
        cls.max = findNearestNeighbours(cls.distances)

    def testDistancesSize(self):
        self.assertEqual(len(self.users), len(self.distances))

    def testIfDistancesNotNone(self):
        for i in self.distances.keys():
            for j in self.distances[i]:
                self.assertIsNotNone(j)
    
    def testIfCalculatesDiagonalDistanceCorrectly(self):
        for i in self.distances.keys():
            for j in self.distances[i]:
                if i == j:
                    self.assertEqual(j, 0)

    def testIfCalulatesProperly(self):
        userss = self.users
        geo1 = (userss[1]['address']['geo']['lat'], userss[1]['address']['geo']['lng'])
        geo2 = (userss[2]['address']['geo']['lat'], userss[2]['address']['geo']['lng'])
        dist = geopy.distance.distance(geo1, geo2).km
        self.assertEqual(dist, self.distances[1][1])
    
    def testIfNearestNeighboursDoesntCalculateZeros(self):
        for i in self.max:
            self.assertNotEqual(self.max[i][1], 0)

    def testIfNearestNeigboursCalculatesProperly(self):
        self.assertEqual(self.max[0][1], 1810.3222631439153)
        self.assertEqual(self.max[1][1], 2841.1566568831986)
        self.assertEqual(self.max[2][1], 2841.1566568831986)

    def testNearestNeighboursLength(self):
        self.assertEqual(len(self.max), len(self.users))

    

Success

.......
----------------------------------------------------------------------
Ran 7 tests in 0.188s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>